# Data to Decision

## Nathan Scheperle
## Monday, February 11

Note: students may work in groups on the problem but are responsible for submitting their own answers. Type answers directly in this word document, rename the file with your name, and upload to Sakai Dropbox before noon on Monday. 

## Applying the Threshold Model and Forecasting Future Error Rates

Computer chips that exceed certain size tolerances on the assembly line tend to be defective. You are given size measures for each chip, wirth binary outcomes (defective/not defective). You are given 400 typical cases (drawn at random from a stable production process). 

The cost per FP classification is estimated at $\$50-100$ and the cost per FN classification is $\$150-300$.  

In [1]:
class Threshold():
    """ Implementation of the Threshold Model in Python"""
    
    def __init__(self, fn_cost, fp_cost):
        self.fn_cost = fn_cost
        self.fp_cost = fp_cost
        pass
    
    def fit(self, x, y):
        df = pd.concat([x, y], axis=1)
        df.columns = ['x', 'y']
        pos = len(df[df['y']])
        neg = len(df) - pos
        df = df.sort_values(by='x', ascending=False)
        if df.iloc[0, df.columns.get_loc('y')]:
            df['FP'] = 0
            df['FN'] = pos - 1
            df['TP'] = 1
            df['TN'] = neg
        else:
            df['FP'] = 1
            df['FN'] = pos
            df['TP'] = 0
            df['TN'] = neg  - 1
        for i in range(1,len(df)):
            if df.iloc[i,df.columns.get_loc('y')]:
                df.iloc[i, df.columns.get_loc('FN')] = df.iloc[i-1, df.columns.get_loc('FN')] - 1
                df.iloc[i, df.columns.get_loc('TP')] = df.iloc[i-1, df.columns.get_loc('TP')] + 1

                df.iloc[i, df.columns.get_loc('TN')] = df.iloc[i-1, df.columns.get_loc('TN')]
                df.iloc[i, df.columns.get_loc('FP')] = df.iloc[i-1, df.columns.get_loc('FP')]
            else:
                df.iloc[i, df.columns.get_loc('TN')] = df.iloc[i-1, df.columns.get_loc('TN')] - 1
                df.iloc[i, df.columns.get_loc('FP')] = df.iloc[i-1, df.columns.get_loc('FP')] + 1

                df.iloc[i, df.columns.get_loc('FN')] = df.iloc[i-1, df.columns.get_loc('FN')]
                df.iloc[i, df.columns.get_loc('TP')] = df.iloc[i-1, df.columns.get_loc('TP')]

        df['Cost'] = self.fn_cost*df['FN'] + self.fp_cost*df['FP']
        df['Error'] = (df['FN'] + df['FP'])/len(df)
        
        threshold_row = df.loc[df['Cost'].idxmin()]
        self.threshold = threshold_row['x']
        self.err_rate = threshold_row['Error']
        self.cost = threshold_row['Cost']
        self.n = len(df)
        pass
    
    def avg_cost(self):
        return self.cost/self.n
    
    def predict(self, x):
        x_sort = x.sort_values(ascending=False)
        y_hat = x >= self.threshold
        return y_hat    
    

Partition your $400$ cases into a *training set* and *test set*, based on ensuring that with $99\%$ confidence the error rate on future data will not exceed the error rate on test data by $.10$.  Partition the data by assigning individual cases at random to the two sets in your desired proportions.

1. How large is your test set? Why? 

Using Hoeffding's Inequality, the required number of observations in the test dataset to ensure an error rate below $.10$ at the $99\%$ confidence level is:
$$
n \geq \frac{log(2/\alpha)}{2t^2} = \frac{log(2/.01)}{2 \cdot .1^2} = \frac{log(200)}{.02} \approx 265
$$

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate

# read in dataset
conv = lambda input: input=='POS'
df = pd.read_excel("./data/Data for Feb 11 Problem.xlsx", header=None, 
                   names=['x','y'])
df['y'] = conv(df['y'])

# n derived above
n = 400 - 265

train_idx = np.random.choice(range(len(df)), size=n, replace=False)

train_df = df.iloc[train_idx]
test_df = df.drop(train_idx)

2. Evaluate the threshold model on your training data only, and identify six parameters (thresholds) that minimize the average per-chip cost of misclassification at each of six combinations of costs. 

(A) Record these parameters. Also give (B) the average cost per event and (C) the error rate at each threshold on training set.

In [3]:
def do_threshold(dataset, fn_cost, fp_cost):
    thresh_model = Threshold(fn_cost, fp_cost)
    thresh_model.fit(dataset['x'], dataset['y'])
    print("At ${} per FN and ${} per FP, ".format(fn_cost, fp_cost))
    print("Threshold = {}".format(thresh_model.threshold))
    print("Error rate = {}".format(thresh_model.err_rate))
    print("Average cost per event = ${0:.4}".format(thresh_model.avg_cost()))
    print()
    return thresh_model, fn_cost, fp_cost, thresh_model.threshold, thresh_model.avg_cost(), thresh_model.err_rate 

train_150_50 = do_threshold(train_df, 150, 50)
model_150_50 = train_150_50[0]
train_225_50 = do_threshold(train_df, 225, 50)
model_225_50 = train_225_50[0]
train_350_50 = do_threshold(train_df, 350, 50)
model_350_50 = train_350_50[0]

train_150_100 = do_threshold(train_df, 150, 100)
model_150_100 = train_150_100[0]
train_225_100 = do_threshold(train_df, 225, 100)
model_225_100 = train_225_100[0]
train_350_100 = do_threshold(train_df, 350, 100)
model_350_100 = train_350_100[0]

At $150 per FN and $50 per FP, 
Threshold = 10219.89413379076
Error rate = 0.037037037037037035
Average cost per event = $3.333

At $225 per FN and $50 per FP, 
Threshold = 10219.89413379076
Error rate = 0.037037037037037035
Average cost per event = $4.444

At $350 per FN and $50 per FP, 
Threshold = 10219.89413379076
Error rate = 0.037037037037037035
Average cost per event = $6.296

At $150 per FN and $100 per FP, 
Threshold = 10241.889257347055
Error rate = 0.02962962962962963
Average cost per event = $4.074

At $225 per FN and $100 per FP, 
Threshold = 10219.89413379076
Error rate = 0.037037037037037035
Average cost per event = $5.556

At $350 per FN and $100 per FP, 
Threshold = 10219.89413379076
Error rate = 0.037037037037037035
Average cost per event = $7.407



In [4]:
headers = ["Cost Per FN", "Cost Per FP", "Threshold", "Average Cost", "Error Rate"]
table = [train_150_50[1:], train_225_50[1:], train_350_50[1:], 
        train_150_100[1:], train_225_100[1:], train_350_100[1:]]

display(HTML(tabulate.tabulate(table, headers=headers, tablefmt='html')))



Cost Per FN,Cost Per FP,Threshold,Average Cost,Error Rate
150,50,10219.9,3.33333,0.037037
225,50,10219.9,4.44444,0.037037
350,50,10219.9,6.2963,0.037037
150,100,10241.9,4.07407,0.0296296
225,100,10219.9,5.55556,0.037037
350,100,10219.9,7.40741,0.037037


3. Evaluate the six thresholds derived from training data on the test data and record (B) the average cost per event and (C) the error rate at each threshold. 

In [5]:
def cost_error(pred_df, fn_cost, fp_cost):
    fn_count = len(pred_df.loc[pred_df['y'] & ~pred_df['predict']])
    fp_count = len(pred_df.loc[~pred_df['y'] & pred_df['predict']])
    
    cost = fn_count*fn_cost + fp_count*fp_cost
    avg_cost = cost/len(pred_df)
    err_rate = (fn_count+fp_count)/len(pred_df)
    return fn_cost, fp_cost, avg_cost, err_rate, fn_count/len(pred_df), fp_count/len(pred_df)
    
df_150_50 = test_df
df_150_50['predict'] = model_150_50.predict(test_df['x'])
df_150_50['predict'] = test_df['x'] >= model_150_50.threshold
test_150_50 = cost_error(df_150_50, 150, 50)

df_225_50 = test_df
df_225_50['predict'] = model_225_50.predict(test_df['x'])
df_225_50['predict'] = test_df['x'] >= model_225_50.threshold
test_225_50 = cost_error(df_225_50, 225, 50)

df_350_50 = test_df
df_350_50['predict'] = model_350_50.predict(test_df['x'])
df_350_50['predict'] = test_df['x'] >= model_350_50.threshold
test_350_50 = cost_error(df_350_50, 350, 50)

df_150_100 = test_df
df_150_100['predict'] = model_150_100.predict(test_df['x'])
df_150_100['predict'] = test_df['x'] >= model_150_100.threshold
test_150_100 = cost_error(df_150_100, 150, 100)

df_225_100 = test_df
df_225_100['predict'] = model_225_100.predict(test_df['x'])
df_225_100['predict'] = test_df['x'] >= model_225_100.threshold
test_225_100 = cost_error(df_225_100, 225, 100)

df_350_100 = test_df
df_350_100['predict'] = model_350_100.predict(test_df['x'])
df_350_100['predict'] = test_df['x'] >= model_350_100.threshold
test_350_100 = cost_error(df_350_100, 350, 100)


headers = ["Cost Per FN", "Cost Per FP", "Average Cost Per Event", "Error Rate"]
table = [test_150_50[:4], test_225_50[:4], test_350_50[:4], 
        test_150_100[:4], test_225_100[:4], test_350_100[:4]]

display(HTML(tabulate.tabulate(table, headers=headers, tablefmt='html')))

Cost Per FN,Cost Per FP,Average Cost Per Event,Error Rate
150,50,9.0566,0.0679245
225,50,13.3019,0.0679245
350,50,20.3774,0.0679245
150,100,8.86792,0.0603774
225,100,13.8679,0.0679245
350,100,20.9434,0.0679245


4. With 99% reliability, what is the expected maximum error rate on new data at each of the six thresholds? 

In [6]:
table = []
for l in [test_150_50, test_225_50, test_350_50, 
        test_150_100, test_225_100, test_350_100]:
    table.append(list(l[0:2]) + [l[3] + .1])
    
headers = ["Cost Per FN", "Cost Per FP", "Max Error Rate"]

display(HTML(tabulate.tabulate(table, headers=headers, tablefmt='html')))

Cost Per FN,Cost Per FP,Max Error Rate
150,50,0.167925
225,50,0.167925
350,50,0.167925
150,100,0.160377
225,100,0.167925
350,100,0.167925


Note that the Hoeffding inequality can be used not only to generalize the overall error rate from test data results, but also the separate overall portion of False Positives (not the FP rate, g/b, but the portion g/(a+b)) and the overall portion of False Negatives (not the FN rate, f/a, but the portion of False Negatives f/(a+b)).  

5. Calculate these portions for each of the six parameters on the test data. 

In [7]:
headers = ["Cost Per FN", "Cost Per FP", "FN Rate", "FP Rate"]
table = [list(test_150_50[0:2])+list(test_150_50[4:]), list(test_225_50[0:2])+list(test_225_50[4:]),
        list(test_350_50[0:2])+list(test_350_50[4:]), 
        list(test_150_100[0:2])+list(test_150_100[4:]), list(test_225_100[0:2])+list(test_225_100[4:]),
              list(test_350_100[0:2])+list(test_350_100[4:])]

display(HTML(tabulate.tabulate(table, headers=headers, tablefmt='html')))

Cost Per FN,Cost Per FP,FN Rate,FP Rate
150,50,0.0566038,0.0113208
225,50,0.0566038,0.0113208
350,50,0.0566038,0.0113208
150,100,0.0566038,0.00377358
225,100,0.0566038,0.0113208
350,100,0.0566038,0.0113208


6. With 99% confidence, what is the maximum portion of False Positives, and the maximum portion of False Negatives, that will be observed at each of the six thresholds? 

In [8]:
headers = ["Cost Per FN", "Cost Per FP", "Max FN Rate", "Max FP Rate"]

table = [list(test_150_50[0:2])+[x+.1 for x in test_150_50[4:]], list(test_225_50[0:2])+[x+.1 for x in test_225_50[4:]],
        list(test_350_50[0:2])+[x+.1 for x in test_350_50[4:]], 
        list(test_150_100[0:2])+[x+.1 for x in test_150_100[4:]], list(test_225_100[0:2])+[x+.1 for x in test_225_100[4:]],
              list(test_350_100[0:2])+[x+.1 for x in test_350_100[4:]]]



display(HTML(tabulate.tabulate(table, headers=headers, tablefmt='html')))

Cost Per FN,Cost Per FP,Max FN Rate,Max FP Rate
150,50,0.156604,0.111321
225,50,0.156604,0.111321
350,50,0.156604,0.111321
150,100,0.156604,0.103774
225,100,0.156604,0.111321
350,100,0.156604,0.111321


7. With 99% confidence, what is the maximum cost per event that will be observed at each of the six thresholds?  

In [9]:
headers = ["Cost Per FN", "Cost Per FP", "Max Cost Per Event"]

table = [list(test_150_50[0:2])+[test_150_50[2]*(1+test_150_50[3])],
         list(test_225_50[0:2])+[test_225_50[2]*(1+test_225_50[3])],
        list(test_350_50[0:2])+[test_350_50[2]*(1+test_350_50[3])], 
        list(test_150_100[0:2])+[test_150_100[2]*(1+test_150_100[3])],
         list(test_225_100[0:2])+[test_225_100[2]*(1+test_225_100[3])],
              list(test_350_100[0:2])+[test_350_100[2]*(1+test_350_100[3])]]

display(HTML(tabulate.tabulate(table, headers=headers, tablefmt='html')))

Cost Per FN,Cost Per FP,Max Cost Per Event
150,50,9.67177
225,50,14.2054
350,50,21.7615
150,100,9.40335
225,100,14.8099
350,100,22.366


**END**